In [28]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.utils import resample
from bayes_opt import BayesianOptimization
import time


In [29]:
X_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/X_train_TS25_RS3.csv')
X_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/X_test_TS25_RS3.csv')
y_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/y_train_TS25_RS3.csv')
y_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/y_test_TS25_RS3.csv')

In [30]:
Narrow_X_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_X_train_TS25_RS3.csv')
Narrow_X_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_X_test_TS25_RS3.csv')
Narrow_y_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_y_train_TS25_RS3.csv')
Narrow_y_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_y_test_TS25_RS3.csv')

In [31]:
print('problemtype: medical : Catch all isntances of event, false positives are less of a problem. Maximize: Sensitivity, and Precision')
print('problemtype: spam: Catch many instances of event, but minimize FP')
print('problem type: Artisinal, run time less important than accuracy')
print('problem type: Enterprise, run time is a priority')

problemtype: medical : Catch all isntances of event, false positives are less of a problem. Maximize: Sensitivity, and Precision
problemtype: spam: Catch many instances of event, but minimize FP
problem type: Artisinal, run time less important than accuracy
problem type: Enterprise, run time is a priority


In [32]:
def timing_buddy(model_to_fit, dictionary_string, dicto):
    funky_time_start = time.time()
    model_to_fit.fit(X_train, np.ravel(y_train))
    funky_time_stop = time.time()
    funky_train_time = funky_time_stop - funky_time_start
    dicto[dictionary_string]['training_time'] = funky_train_time
    

In [33]:
def confused_buddy(model_to_confuse, dictionary_string,dicto):
    confuse = confusion_matrix(y_test, model_to_confuse.predict(X_test))
    dicto[dictionary_string]['confuse'] = confuse

## Dummy

In [34]:
def classy_dummy(dicto):
    dummy_clf = DummyClassifier(strategy="most_frequent")
    timing_buddy(dummy_clf, 'dummy', dicto)
    confused_buddy(dummy_clf, 'dummy', dicto)
    confused_buddy(dummy_clf, 'dummy', dicto)
    dicto['dummy']['ROC_AUC_Score'] = roc_auc_score(y_test, dummy_clf.predict_proba(X_test)[:, 1])
    

## SVC

In [35]:
def SVCmodeler(dicto):
    SupportVectorClassifier = SVC()
    timing_buddy(SupportVectorClassifier, 'SVC', dicto)
    confused_buddy(SupportVectorClassifier, 'SVC', dicto)
    dicto['SVC']['ROC_AUC_Score'] = roc_auc_score(y_test, SupportVectorClassifier.decision_function(X_test))


## Stochastic Gradiant Descent 

In [36]:
def SGDModeler(dicto):
    SGD_clf = SGDClassifier()
    timing_buddy(SGD_clf, 'SGDClassfier', dicto)
    confused_buddy(SGD_clf, 'SGDClassfier', dicto)
    dicto['SGDClassfier']['ROC_AUC_Score'] = roc_auc_score(y_test, SGD_clf.decision_function(X_test))

## SGD Hyper Paramter optimization

In [ ]:
def SGDHyperParameterer(loss_penalty_list):
    SGD_clf = SGDClassifier(loss = loss_penalty_list[0], penalty=loss_penalty_list[1])
    confuse = confusion_matrix(y_test, SGD_clf.predict(X_test))
    tp = confuse[0][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    return recall
    
    

In [ ]:
losses= ['modified_huber', 'log']
penalties = ['l2', 'l1', 'elasticnet']
param_grid = {}
count = 0
for each_loss in losses:
    for each_penalty in penalties:
        param_grid[str(each_loss) + '_' + str(each_penalty)] = [each_loss, each_penalty]
        count+=1

In [ ]:
SGD_Hyper_param_recall_score
for items in param_grid:
    

## BaggingClassifier

In [37]:
def BagginsesModeler(dicto):
    Classy_Baggins_DT = BaggingClassifier(n_estimators=10)
    timing_buddy(Classy_Baggins_DT, 'BaggingClassifier', dicto)
    confused_buddy(Classy_Baggins_DT, 'BaggingClassifier', dicto)
    dicto['BaggingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, Classy_Baggins_DT.predict_proba(X_test)[:, 1])

## RandomForestClassifier

In [38]:
def RFModeler(dicto):
    RandoForest = RandomForestClassifier(max_depth=110, min_samples_split = 1, min_samples_leaf=2, max_features='sqrt', n_estimators=1244)
    
    timing_buddy(RandoForest, 'RandomForestClassifier', dicto)
    confused_buddy(RandoForest, 'RandomForestClassifier', dicto)
    dicto['RandomForestClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, RandoForest.predict_proba(X_test)[:, 1])

## random forest bayes hyper parameter tuning

In [83]:
model_set = ['dummy','SVC', 'SGDClassfier', 'BaggingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'HistGradientBoostingClassifier']
dicto = {}
for each in model_set:
    dicto[each] ={'confuse' : [], 'training_time' : 0, 'ROC_AUC_Score': 0}

In [2]:
def bayes_RFModeler_auto(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    RandoForest = RandomForestClassifier(n_estimators=int(n_estimators), max_features='auto', max_depth=int(max_depth),
                                         min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf))
    RandoForest.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, RandoForest.predict(X_test))
    
    tp = confuse[0][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    
    return recall

In [93]:
def bayes_RFModeler_sqrt(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    RandoForest = RandomForestClassifier(n_estimators=int(n_estimators), max_features='sqrt', max_depth=int(max_depth),
                                         min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf))
    RandoForest.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, RandoForest.predict(X_test))
    tp = confuse[0][0]
    fp = confuse[1][0]
    precision = tp/(tp + fp)

    return precision

In [94]:
# Number of trees in random forest
param_dicts = { 'n_estimators' : (100, 2000),
              'max_depth' : (10,110),
              'min_samples_split': (2,10),
              'min_samples_leaf' : (1,5),
              }
#'max_features' : ('auto', 'sqrt'),


print(param_dicts)

{'n_estimators': (100, 2000), 'max_depth': (10, 110), 'min_samples_split': (2, 10), 'min_samples_leaf': (1, 5)}


In [95]:
optimizer_auto = BayesianOptimization(
    bayes_RFModeler_auto,
    pbounds=param_dicts
)

In [96]:
bayes_sampling_time_start= time.time()
optimizer_auto.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.8292   |  99.58    |  4.138    |  2.99     |  1.808e+0 |
|  2        |  0.8299   |  86.24    |  2.283    |  5.452    |  999.6    |
|  3        |  0.8297   |  102.1    |  3.114    |  8.573    |  1.154e+0 |
|  4        |  0.8284   |  29.15    |  4.801    |  8.869    |  1.491e+0 |
|  5        |  0.8292   |  53.33    |  4.411    |  3.588    |  213.2    |
|  6        |  0.8293   |  63.1     |  3.077    |  5.858    |  1.467e+0 |
|  7        |  0.8255   |  17.2     |  3.754    |  2.215    |  1.417e+0 |
|  8        |  0.8288   |  90.69    |  4.306    |  4.015    |  260.5    |
|  9        |  0.8292   |  50.28    |  4.898    |  9.399    |  1.979e+0 |
|  10       |  0.8324   |  62.23    |  1.192    |  3.572    |  772.2    |
|  11       |  0.8307   |  63.11    |  2.025    |  9.033    |  1.947e+0 |
|  12       |  0.8321   |  98.35    | 

In [97]:
optimizer_sqrt = BayesianOptimization(
    bayes_RFModeler_sqrt,
    pbounds=param_dicts
)

In [98]:
bayes_sampling_time_start= time.time()
optimizer_sqrt.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.8285   |  93.51    |  4.937    |  5.694    |  568.4    |
|  2        |  0.83     |  25.34    |  1.639    |  8.309    |  451.9    |
|  3        |  0.8292   |  70.55    |  4.098    |  4.483    |  665.9    |
|  4        |  0.83     |  58.23    |  3.88     |  5.545    |  1.828e+0 |
|  5        |  0.8218   |  15.52    |  4.54     |  5.521    |  715.8    |
|  6        |  0.8275   |  22.09    |  3.268    |  9.804    |  1.975e+0 |
|  7        |  0.8285   |  95.41    |  4.049    |  8.341    |  131.5    |
|  8        |  0.8276   |  23.1     |  3.678    |  5.599    |  1.916e+0 |
|  9        |  0.8308   |  71.11    |  2.997    |  9.542    |  1.836e+0 |
|  10       |  0.8237   |  16.68    |  3.807    |  4.496    |  1.08e+03 |
|  11       |  0.8326   |  96.26    |  1.02     |  2.78     |  1.286e+0 |
|  12       |  0.8128   |  12.17    | 

In [102]:
print((bayes_sampling_time_stop - bayes_sampling_time_start)/60)

48.69563031991323


## AdaBoostClassifier

In [39]:
def AdaBoostModeler(dicto):
    adaira_the_classifier = AdaBoostClassifier(n_estimators=100)
    timing_buddy(adaira_the_classifier, 'AdaBoostClassifier', dicto)
    confused_buddy(adaira_the_classifier, 'AdaBoostClassifier', dicto)
    dicto['AdaBoostClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, adaira_the_classifier.predict_proba(X_test)[:, 1])

## GradientBoostingClassifier

In [40]:
def GradBoostModeler(dicto):
    grady_the_boosted = GradientBoostingClassifier(n_estimators=100)
    timing_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    confused_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    dicto['GradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, grady_the_boosted.predict_proba(X_test)[:, 1])

## HistGradientBoostingClassifier


In [41]:
#for some reason this model didn't like my functions. So long form it is!
def HistGrad_modeler(dicto):
    histamine_reaction = HistGradientBoostingClassifier()

    hist_time_start = time.time()
    histamine_reaction.fit(X_train, np.ravel(y_train))
    hist_time_stop = time.time()
    hist_train_time = hist_time_stop - hist_time_start
    dicto['HistGradientBoostingClassifier']['training_time'] = hist_train_time
    confuse = confusion_matrix(y_test, histamine_reaction.predict(X_test))
    dicto['HistGradientBoostingClassifier']['confuse'] = confuse
    dicto['HistGradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, histamine_reaction.predict_proba(X_test)[:, 1])    


## Model Evaluation

#precision = tp/(tp+fp)
Maximize this for medical or security scenarios, don't wanna miss actual sicknesses

#recall= tp/(tp+fn) 
maximize this for spam, don't wanna be hiding real emails

specificity(tn/(tn+fp)

In [42]:
def medical_evaluator(dicto):
    bestmodel = ""
    training_time= 0
    precision = 0
    for key, value in dicto.items():
        for underkey, under_value in value.items():
            tp = dicto[key]['confuse'][0][0]
            fp = dicto[key]['confuse'][1][0]
            temp_precision = tp/(tp + fp)
            if temp_precision > precision:
                precision = temp_precision
                bestmodel = key
                training_time = dicto[key]['training_time']
        return bestmodel, precision, training_time
            
            

In [43]:
def spam_evaluator(dicto):
    bestmodel = ""
    training_time= 0
    recall = 0
    for key, value in dicto.items():
        for underkey, under_value in value.items():
            tp = dicto[key]['confuse'][0][0]
            fn = dicto[key]['confuse'][0][1]
            temp_recall = tp/(tp + fn)
            if temp_recall > recall:
                recall = temp_recall
                bestmodel = key
                training_time = dicto[key]['training_time']
        return bestmodel, recall, training_time

In [44]:
def chonky_model_aggregator(X_train, y_train, X_test, y_test):
    
    #Models to explore
    model_set = ['dummy','SVC', 'SGDClassfier', 'BaggingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'HistGradientBoostingClassifier']
    model_stats = {}
    for each in model_set:
        model_stats[each] ={'confuse' : [], 'training_time' : 0, 'ROC_AUC_Score': 0}
    #model running
    classy_dummy(model_stats)
    SVCmodeler(model_stats)
    SGDModeler(model_stats)
    BagginsesModeler(model_stats)
    RFModeler(model_stats)
    AdaBoostModeler(model_stats)
    GradBoostModeler(model_stats)
    HistGrad_modeler(model_stats)


    #best_med_model, med_precision, med_time = medical_evaluator(model_stats)
    #best_spam_model, spam_recall, spam_time = spam_evaluator(model_stats)
    #print("The best medical model (Optimizing for precision) is " + best_med_model + " It's precision score was" + str(round(med_precision,3)) + " and it took " + str(round(med_time,2)) +" seconds to run.")
    #print("")
    #print("The best spam model (Optimizing for recall) is " + best_spam_model + " It's recall score was" +str(round(spam_recall, 3)) +" and it took " + str(round(spam_time, 2) +" seconds to run.")
    return model_stats



#Wide Data

In [18]:
wide_data_model_stats = chonky_model_aggregator(X_train, y_train, X_test, y_test)

The best medical model (Optimizing for precision) is dummy It's precision score was0.8037328238665589 and it took 0.017609596252441406 seconds to run.

The best spam model (Optimizing for recall) is dummy It's recall score was1.0 and it took 0.017609596252441406 seconds to run.


In [39]:
for key, value in wide_data_model_stats.items():
    print(key)
    print(value['confuse'])

dummy
[[10938     0]
 [ 2671     0]]
SVC
[[10680   258]
 [ 2200   471]]
SGDClassfier
[[10834   104]
 [ 2426   245]]
BaggingClassifier
[[10377   561]
 [ 2056   615]]
RandomForestClassifier
[[10938     0]
 [ 2671     0]]
AdaBoostClassifier
[[10508   430]
 [ 1990   681]]
GradientBoostingClassifier
[[10616   322]
 [ 2090   581]]
HistGradientBoostingClassifier
[[10500   438]
 [ 2011   660]]


#Narrow Data

In [19]:
narrow_data_model_stats = chonky_model_aggregator(Narrow_X_train, Narrow_y_train, Narrow_X_test, Narrow_y_test)

The best medical model (Optimizing for precision) is dummy It's precision score was0.8037328238665589 and it took 0.023005008697509766 seconds to run.

The best spam model (Optimizing for recall) is dummy It's recall score was1.0 and it took 0.023005008697509766 seconds to run.


In [40]:
for key, value in narrow_data_model_stats.items():
    print(key)
    print(value['confuse'])

dummy
[[10938     0]
 [ 2671     0]]
SVC
[[10680   258]
 [ 2200   471]]
SGDClassfier
[[10890    48]
 [ 2532   139]]
BaggingClassifier
[[10390   548]
 [ 2049   622]]
RandomForestClassifier
[[10938     0]
 [ 2671     0]]
AdaBoostClassifier
[[10508   430]
 [ 1990   681]]
GradientBoostingClassifier
[[10616   322]
 [ 2090   581]]
HistGradientBoostingClassifier
[[10485   453]
 [ 1992   679]]


In [20]:
#That didn't go well, so we're going to play with oversampling our smokers in both sets

In [21]:
Xy_to_oversample= X_train.join(y_train)

In [24]:
Xy_mask = Xy_to_oversample.smokes == 'yes'
Xy_oversampled = resample(Xy_to_oversample[Xy_mask], n_samples=20000, replace=True)

In [25]:
oversampled_X_train = Xy_oversampled.drop('smokes', axis=1)    
oversampled_y_train = Xy_oversampled.smokes

#resampled Wide Data

In [26]:
wide_data_model_stats = chonky_model_aggregator(oversampled_X_train, oversampled_y_train, X_test, y_test)

The best medical model (Optimizing for precision) is dummy It's precision score was0.8037328238665589 and it took 0.016995906829833984 seconds to run.

The best spam model (Optimizing for recall) is dummy It's recall score was1.0 and it took 0.016995906829833984 seconds to run.


In [36]:
for key, value in wide_data_model_stats.items():
    print(key)a
    print(value['confuse'])

dummy
[[10938     0]
 [ 2671     0]]
SVC
[[10680   258]
 [ 2200   471]]
SGDClassfier
[[10834   104]
 [ 2426   245]]
BaggingClassifier
[[10377   561]
 [ 2056   615]]
RandomForestClassifier
[[10938     0]
 [ 2671     0]]
AdaBoostClassifier
[[10508   430]
 [ 1990   681]]
GradientBoostingClassifier
[[10616   322]
 [ 2090   581]]
HistGradientBoostingClassifier
[[10500   438]
 [ 2011   660]]


#Resampled Narrow Data

In [27]:
Narrow_Xy_to_oversample= Narrow_X_train.join(Narrow_y_train)

In [28]:
Narrow_Xy_mask = Narrow_Xy_to_oversample.smokes == 'yes'
Narrow_Xy_oversampled = resample(Narrow_Xy_to_oversample[Narrow_Xy_mask], n_samples=20000, replace=True)

In [29]:
Narrow_oversampled_X_train = Narrow_Xy_oversampled.drop('smokes', axis=1)    
Narrow_oversampled_y_train = Narrow_Xy_oversampled.smokes

In [30]:
narrow_resampled_data_model_stats = chonky_model_aggregator(Narrow_oversampled_X_train, Narrow_oversampled_y_train, Narrow_X_test, Narrow_y_test)

The best medical model (Optimizing for precision) is dummy It's precision score was0.8037328238665589 and it took 0.017003536224365234 seconds to run.

The best spam model (Optimizing for recall) is dummy It's recall score was1.0 and it took 0.017003536224365234 seconds to run.


In [37]:
for key, value in narrow_resampled_data_model_stats.items():
    print(key)
    print(value['confuse'])

dummy
[[10938     0]
 [ 2671     0]]
SVC
[[10680   258]
 [ 2200   471]]
SGDClassfier
[[10876    62]
 [ 2510   161]]
BaggingClassifier
[[10342   596]
 [ 2061   610]]
RandomForestClassifier
[[10938     0]
 [ 2671     0]]
AdaBoostClassifier
[[10508   430]
 [ 1990   681]]
GradientBoostingClassifier
[[10616   322]
 [ 2090   581]]
HistGradientBoostingClassifier
[[10530   408]
 [ 2014   657]]


In [21]:
pbounds = {'samples': (500,50000)}

In [45]:
def precise_sampling_bayes_box(samples):
    
    Xy_to_oversample= X_train.join(y_train)
    Xy_mask = Xy_to_oversample.smokes == 'yes'
    Xy_oversampled = resample(Xy_to_oversample[Xy_mask], n_samples=int(samples), replace=True)
    oversampled_X_train = Xy_oversampled.drop('smokes', axis=1)    
    oversampled_y_train = Xy_oversampled.smokes
    
    
    
    
    #testing the data
    model_stats = chonky_model_aggregator(oversampled_X_train, oversampled_y_train, X_test, y_test)
    best_med_model, med_precision, med_time = medical_evaluator(model_stats)
    return med_precision
    

In [46]:
optimizer = BayesianOptimization(
    precise_sampling_bayes_box,
    pbounds=pbounds
)

In [47]:
bayes_sampling_time_start= time.time()
optimizer.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   |  samples  |
-------------------------------------
|  1        |  0.8037   |  4.372e+0 |
|  2        |  0.8037   |  4.173e+0 |
|  3        |  0.8037   |  9.848e+0 |
|  4        |  0.8037   |  3.144e+0 |
|  5        |  0.8037   |  2.488e+0 |
|  6        |  0.8037   |  9.076e+0 |
|  7        |  0.8037   |  5.892e+0 |
|  8        |  0.8037   |  3.039e+0 |
|  9        |  0.8037   |  3.206e+0 |
|  10       |  0.8037   |  3.786e+0 |
|  11       |  0.8037   |  1.599e+0 |
|  12       |  0.8037   |  2.023e+0 |
|  13       |  0.8037   |  4.037e+0 |
|  14       |  0.8037   |  1.797e+0 |
|  15       |  0.8037   |  1.017e+0 |
|  16       |  0.8037   |  4.655e+0 |
|  17       |  0.8037   |  5e+04    |
|  18       |  0.8037   |  503.9    |
|  19       |  0.8037   |  5e+04    |
|  20       |  0.8037   |  2.743e+0 |
|  21       |  0.8037   |  502.1    |
|  22       |  0.8037   |  3.547e+0 |
|  23       |  0.8037   |  4.726e+0 |
|  24       |  0.8037   |  7.469e+0 |
|  25       

In [52]:
print(((bayes_sampling_time_stop- bayes_sampling_time_start)/60)/60)

4.081936113701926
